# CNN v3 SUPER - Entrenamiento Intensivo 🔥

Modelo más profundo, más épocas, TODO el dataset.

**Setup:**
1. Runtime > Change runtime type > **GPU (T4)**
2. Run all cells
3. Espera ~30-40 minutos
4. Descarga el `.onnx` al final

**Objetivo:** >90% accuracy en EMNIST

In [ ]:
# 1. Install Dependencies
!pip install -q torch torchvision torchaudio albumentations onnx onnxruntime onnxscript


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 108.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 137.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.0/683.0 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.6 MB/s eta 0:00:00


In [ ]:
# 2. Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import EMNIST
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm.notebook import tqdm
import json
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎮 Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

🎮 Device: cuda
   GPU: Tesla T4


In [ ]:
# 3. SUPER Model Architecture (Deeper ResNet)
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class CharCNN_v3_SUPER(nn.Module):
    """Deeper model with more capacity"""
    def __init__(self, num_classes=62, dropout_rate=0.4):
        super(CharCNN_v3_SUPER, self).__init__()
        
        # Initial conv
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        # Deeper ResNet blocks
        self.layer1 = self._make_layer(64, 64, 2, stride=1)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)
        self.layer4 = self._make_layer(256, 512, 2, stride=2)  # Extra layer
        
        # Classifier
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.5),
            nn.Linear(256, num_classes)
        )

    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels, 1))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = CharCNN_v3_SUPER().to(device)
print(f"🎯 Parameters: {sum(p.numel() for p in model.parameters()):,}")

🎯 Parameters: 11,314,942


In [ ]:
# 4. Data Augmentation with Albumentations
train_transform = A.Compose([
    A.ElasticTransform(p=0.3),
    A.GridDistortion(p=0.2),
    A.Affine(rotate=(-15, 15), translate_percent=0.1, scale=(0.9, 1.1), p=0.5),
    A.GaussNoise(var_limit=(5, 20), p=0.2),
])

class EMNISTAugmented(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        img_np = np.array(img).transpose()  # Fix EMNIST orientation
        
        if self.transform:
            augmented = self.transform(image=img_np)
            img_np = augmented['image']
        
        img_tensor = torch.from_numpy(img_np).float() / 255.0
        img_tensor = img_tensor.unsqueeze(0)
        return img_tensor, label

print("📥 Downloading EMNIST (full dataset)...")
train_dataset_raw = EMNIST(root='./data', split='byclass', train=True, download=True)
test_dataset_raw = EMNIST(root='./data', split='byclass', train=False, download=True)

train_dataset = EMNISTAugmented(train_dataset_raw, transform=train_transform)
test_dataset = EMNISTAugmented(test_dataset_raw, transform=None)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=2)

print(f"📊 Train: {len(train_dataset):,} | Test: {len(test_dataset):,}")

/tmp/ipython-input-3996947078.py:6: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5, 20), p=0.2),


📥 Downloading EMNIST (full dataset)...


100%|██████████| 562M/562M [00:06<00:00, 88.1MB/s] 


📊 Train: 697,932 | Test: 116,323


In [ ]:
# 5. Training Setup
EPOCHS = 30
LEARNING_RATE = 0.001

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Label smoothing for better generalization
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=0.01,
    epochs=EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.3
)

print(f"📈 Training for {EPOCHS} epochs with OneCycleLR")

📈 Training for 30 epochs with OneCycleLR


In [ ]:
# 6. Training Loop
best_acc = 0.0
history = {'train_loss': [], 'train_acc': [], 'test_acc': []}

for epoch in range(EPOCHS):
    # Training
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'loss': f"{loss.item():.4f}",
            'acc': f"{100.*train_correct/train_total:.2f}%",
            'lr': f"{scheduler.get_last_lr()[0]:.6f}"
        })
    
    train_acc = 100. * train_correct / train_total
    
    # Validation
    model.eval()
    test_correct = 0
    test_total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Validation", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()
    
    test_acc = 100. * test_correct / test_total
    
    history['train_loss'].append(train_loss / len(train_loader))
    history['train_acc'].append(train_acc)
    history['test_acc'].append(test_acc)
    
    print(f"\n📊 Epoch {epoch+1}: Train={train_acc:.2f}% | Test={test_acc:.2f}%")
    
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), 'best_model_v3_super.pth')
        print(f"✅ New best: {test_acc:.2f}%")

print(f"\n🎉 Training complete! Best: {best_acc:.2f}%")

Epoch 1/30:   0%|          | 0/2727 [00:00<?, ?it/s]

Validation:   0%|          | 0/455 [00:00<?, ?it/s]


📊 Epoch 1: Train=81.89% | Test=85.80%
✅ New best: 85.80%


Epoch 2/30:   0%|          | 0/2727 [00:00<?, ?it/s]

Validation:   0%|          | 0/455 [00:00<?, ?it/s]

...


📊 Epoch 4: Train=85.16% | Test=85.96%


Epoch 5/30:   0%|          | 0/2727 [00:00<?, ?it/s]

Validation:   0%|          | 0/455 [00:00<?, ?it/s]


📊 Epoch 5: Train=85.30% | Test=86.51%


Epoch 6/30:   0%|          | 0/2727 [00:00<?, ?it/s]

In [ ]:
# 7. Save Config
config = {
    'num_classes': 62,
    'dropout_rate': 0.4,
    'model_version': 'v3_super',
    'best_test_acc': best_acc,
    'epochs_trained': EPOCHS,
    'architecture': 'ResNet-SUPER (8 blocks)',
    'parameters': sum(p.numel() for p in model.parameters())
}

with open('config_v3_super.json', 'w') as f:
    json.dump(config, f, indent=4)

print("💾 Config saved")
print(json.dumps(config, indent=2))

In [ ]:
# 8. Export to ONNX
import os

print("🔄 Exporting to ONNX...")

model_path = 'best_model_v3_super.pth'
if not os.path.exists(model_path):
    raise FileNotFoundError(f"❌ No se encontró el archivo {model_path}. Asegúrate de que el entrenamiento haya terminado correctamente.")

# Load best model
print(f"   Loading model from {model_path}...")
model.load_state_dict(torch.load(model_path))
model.eval()

# Dummy input on GPU (device)
dummy_input = torch.randn(1, 1, 28, 28).to(device)

try:
    torch.onnx.export(
        model,
        dummy_input,
        'model_v3_super.onnx',
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    print("✅ ONNX exported successfully!")
except Exception as e:
    print(f"❌ Error exporting to ONNX: {e}")
    raise e

# Test ONNX
print("   Verifying ONNX model...")
import onnxruntime as ort
try:
    # ONNX Runtime usually runs on CPU by default unless configured otherwise, which is fine for verification
    session = ort.InferenceSession('model_v3_super.onnx')
    test_input = np.random.randn(1, 1, 28, 28).astype(np.float32)
    outputs = session.run(None, {'input': test_input})
    print(f"✅ ONNX verified: output shape {outputs[0].shape}")
except Exception as e:
    print(f"❌ Error verifying ONNX model: {e}")

In [ ]:
# 9. Download Files
from google.colab import files

print("📥 Descargando modelo ONNX...")
files.download('model_v3_super.onnx')
files.download('config_v3_super.json')

print("\n✅ Archivos descargados!")
print("\n📌 Instrucciones:")
print("1. Coloca 'model_v3_super.onnx' en: models/cnn_modelo_v3/model.onnx")
print("2. Coloca 'config_v3_super.json' en: models/cnn_modelo_v3/config_v3.json")
print("3. La API cargará automáticamente el nuevo modelo!")
print(f"\n🎯 Accuracy final: {best_acc:.2f}%")

## 🎉 ¡Listo!

Modelo **v3 SUPER** entrenado con:
- Arquitectura más profunda (ResNet con 8 bloques)
- Data augmentation avanzado
- OneCycleLR para mejor convergencia
- Label smoothing
- TODO el dataset EMNIST

**Próximos pasos:**
1. Descarga los archivos
2. Colócalos en tu proyecto local
3. ¡Usa el modelo en la API! 🚀